In [1]:
import names
import random as rand
import numpy as np
from math import floor
from datetime import datetime as dt

from _1_Identifiers import Identifiers 
from _2_Characteristics import Characteristics
from _3_Diagnosis import Diagnosis
from _4_KidneyFunction import KidneyFunction
from _5_LoadDose import LoadDose

from _7_PopulationBasedPKParameters import PopulationBasedPKParameters

rand.seed(1)
np.random.seed(1)

In [2]:
import math

class MaintenanceDose:
    calcTDD = 0
    MDDose = 0
    MDTinf = 0
    tau = 0
    
    def __init__(self, pt, MDDose = 0, MDTinf = 0, tau=0, default = False, calcTDD = 0):
        self.pt = pt
        
        self.MDDose = MDDose
        self.MDTinf = MDTinf
        self.tau = tau
        
        if default:
            self.calcTDD = int(500 * pt.pk_pop.CLVanco())
            
            def findRegimen():
                def tinf(strength):
                    if strength in [500]:
                        return 0.5
                    if strength in [750, 1000]:
                        return 1
                    if strength in [1250, 1500]:
                        return 1.5
                    if strength in [1750, 2000]:
                        return 2

                minLoss = 500
                minDose = {}

                for strength in [500,750,1000,1250,1500,1750,2000]:
                    for tau in [6,8,12,18,24,36,48]:
                        dose = {
                            'dose': strength,
                            'tinf': tinf(strength),
                            'tau': tau
                        }
                        popAUC = pt.pk_pop.AUC(**dose)
                        loss = abs(popAUC-500)
                        if tau in [6,18,36,48]:
                            loss += 50
                        loss += 2 * abs(tau-self.pt.pk_pop.t1_2())
                        if loss < minLoss:
                            minLoss = loss
                            minDose = dose

                self.MDDose = minDose['dose']
                self.MDTinf = minDose['tinf']
                self.tau = minDose['tau']
            findRegimen()
        
    def regimen(self):
        if self.MDDose == 0 or self.MDTinf == 0 or self.tau == 0:
            return ""
        else:
            return f"{self.MDDose} mg q{self.tau}h"
    
    def popAUC(self):
        if self.MDDose == 0 or self.MDTinf == 0 or self.tau == 0:
            return 0
        else:
            dose = {
                'dose': self.MDDose,
                'tinf': self.MDTinf,
                'tau': self.tau
            }
            return self.pt.pk_pop.AUC(**dose)


In [3]:
class Patient:
    
    def __init__(self, random = False, id = None):
        if random:
            self.id = Identifiers(random=True)
            self.char = Characteristics(random=True)
            self.diag = Diagnosis(random=True)
            self.kidney = KidneyFunction(self,random=True)
            self.ld = LoadDose(self, default = True)
            self.pk_pop = PopulationBasedPKParameters(self)
            self.md = MaintenanceDose(self, MDDose=1000, MDTinf=1, tau=12, default = True)
        else:
            self.id = Identifiers()
            self.char = Characteristics()
            self.diag = Diagnosis()
            self.kidney = KidneyFunction(self)
            self.ld = LoadDose(self)
            self.pk_pop = PopulationBasedPKParameters(self)
            self.md = MaintenanceDose(self)
    
    def context(self):
        dose = {
            'dose': 1000,
            'tinf': 1,
            'tau':12
        }
        
        print(f"""
      MRN: {self.id.MRN}
     Name: {self.id.LastName}, {self.id.FirstName}
 Location: {self.id.Service} {self.id.Room}

  CLVanco: {self.pk_pop.CLVanco():.1f} L/hr
  calcTDD: {self.md.calcTDD}
  
      Age: {self.char.Age}{self.char.Gender} 
       Wt: {self.char.weightInLbsRoundedWithDescriptor()}
       Ht: {self.char.heightInFeetAndInches()}

      TBW: {self.char.TBW():.1f}
      IBW: {self.char.IBW():.1f}
    AdjBW: {self.char.AdjBW():.1f}

       BMI: {self.char.BMI():.1f}   Obese: {self.char.obeseQ()}
   TBW/IBW: {self.char.TBW_IBW():.1f}    >1.3?: {self.char.TBW_IBW_13Q()}
        
      SCr: {self.kidney.SCr:.2f} mg/dL
   WtUsed: {self.kidney.WeightUsed():.1f}
     CrCl: {self.kidney.CrCl():.1f} mL/min

       Dx: {self.diag.SuspectedIndication}
       Dr: {self.diag.ConsultingPhysician}
      MIC: {self.diag.MIC}

   givenQ: {self.ld.givenQ}
   calcLD: {self.ld.CalculatedLD()}
     Load: {self.ld.LDDose():,} mg over {self.ld.LDTinf()} hrs        
        
       Ke: {self.pk_pop.Ke():.4f}
     t1/2: {self.pk_pop.t1_2():.1f} hrs
    Vd_Kg: {self.pk_pop.Vd_kg()}
       Vd: {self.pk_pop.Vd():.1f} L
  EqnUsed: {self.pk_pop.CLVancoEqnUsed()}
  CLVanco: {self.pk_pop.CLVanco():.1f} L/hr
    
     Cmax: {self.pk_pop.Cmax(**dose):.1f}
     Ceoi: {self.pk_pop.Ceoi(**dose):.1f}
     Cmin: {self.pk_pop.Cmin(**dose):.1f}
      AUC: {self.pk_pop.AUC(**dose)}      
  
  Regimen: {self.md.regimen()}
   popAUC: {self.md.popAUC()}

""")
        


In [4]:
pt = Patient(random=True)
pt.context()


      MRN: 662434
     Name: Carter, Thomas
 Location: 3NSD 390

  CLVanco: 3.2 L/hr
  calcTDD: 1621
  
      Age: 44F 
       Wt: 152.8 lbs
       Ht: 5'6"

      TBW: 69.4
      IBW: 60.4
    AdjBW: 64.0

       BMI: 24.3   Obese: False
   TBW/IBW: 1.1    >1.3?: False
        
      SCr: 0.94 mg/dL
   WtUsed: 60.4
     CrCl: 73.1 mL/min

       Dx: PNA
       Dr: Adreas Vesalius
      MIC: 1

   givenQ: True
   calcLD: 1735
     Load: 1,750 mg over 2 hrs        
        
       Ke: 0.0718
     t1/2: 9.6 hrs
    Vd_Kg: 0.65
       Vd: 45.1 L
  EqnUsed: Matzke
  CLVanco: 3.2 L/hr
    
     Cmax: 41.2
     Ceoi: 38.4
     Cmin: 17.4
      AUC: 638      
  
  Regimen: 750 mg q12h
   popAUC: 479




In [ ]:
      Age: {self.char.Age}{self.char.Gender} 
       Wt: {self.char.weightInLbsRoundedWithDescriptor()}
       Ht: {self.char.heightInFeetAndInches()}

      TBW: {self.char.TBW():.1f}
      IBW: {self.char.IBW():.1f}
    AdjBW: {self.char.AdjBW():.1f}

       BMI: {self.char.BMI():.1f}   Obese: {self.char.obeseQ()}
   TBW/IBW: {self.char.TBW_IBW():.1f}    >1.3?: {self.char.TBW_IBW_13Q()}
        
      SCr: {self.kidney.SCr:.2f} mg/dL
   WtUsed: {self.kidney.WeightUsed():.1f}
     CrCl: {self.kidney.CrCl():.1f} mL/min

       Dx: {self.diag.SuspectedIndication}
       Dr: {self.diag.ConsultingPhysician}
      MIC: {self.diag.MIC}

   givenQ: {self.ld.givenQ}
   calcLD: {self.ld.CalculatedLD()}
     Load: {self.ld.LDDose():,} mg over {self.ld.LDTinf()} hrs        
        
       Ke: {self.pk_pop.Ke():.4f}
     t1/2: {self.pk_pop.t1_2():.1f} hrs
    Vd_Kg: {self.pk_pop.Vd_kg()}
       Vd: {self.pk_pop.Vd():.1f} L
  EqnUsed: {self.pk_pop.CLVancoEqnUsed()}
  CLVanco: {self.pk_pop.CLVanco():.1f} L/hr
    
     Cmax: {self.pk_pop.Cmax(**dose):.1f}
     Ceoi: {self.pk_pop.Ceoi(**dose):.1f}
     Cmin: {self.pk_pop.Cmin(**dose):.1f}
      AUC: {self.pk_pop.AUC(**dose)}    